# LSTM结构和隐藏状态

因为包含循环, 所以循环网络 (RNN) 可以在处理新输入的同时存储信息。长短记忆网络（LSTM）是一种特殊的循环网络，对 LSTM 来说，序列中的每个数据（例如，给定句子中的单词）都有一个相应的 *隐藏状态* $h_t$。该隐藏状态原则上可以包含序列当前结点之前的任一节点的信息；包括一些权重，短期和长期记忆成分的表示。

因此，**LSTM的隐藏状态将根据输入语句中的每个新单词的变化而变化，可以使用隐藏状态来预测语句中下一个最可能的单词**，或者帮助确定语言模型中单词的类型，还有很多其他应用！

### 练习库

请注意，大多数练习的 notebook 都可以根据 [Github练习库](https://github.com/udacity/CVND_Exercises)中的说明在本地计算机上运行。


## Pytorch中的LSTM

要创建和训练 LSTM，必须要知道怎样构造 LSTM 的输入和隐藏状态。在 Pytorch中，可以把 LSTM 定义为：`lstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=n_layers)`。

PyTorch中的LSTM期望的输入都是3D张量，其维度定义如下
>* `input_dim` = 输入中期望的序列的数量 (值为20表示有20个输入序列)
>* `hidden_dim` = 隐藏状态的特征数；即各LSTM单元在每个时间步上的输出的数量。
>* `n_layers ` = 循环层数；通常为1到3之间的值； 值为1表示每个LSTM单元有一个隐藏状态。 默认值为1。

<img src='images/lstm_simple_ex.png' height=5 >
    
###  隐藏状态

定义了 LSTM 的输入和隐藏状态的特征数，我们就可以在每个时间步中用它来获取输出和隐藏状态。 `out, hidden = lstm(input.view(1, 1, -1), (h0, c0))` 

LSTM的输入是 **`(input, (h0, c0))`**.
>* `input` = 包含输入序列特征的张量；张量形状是（seq_len，batch，input_size）
>* `h0` = 包含一个批次中每个元素的初始隐藏状态的张量
>* `c0` = 包含一个批次中每个元素的初始记忆单元的张量


`h0` 和 `c0` 的默认值是 0。其张量形状是： (n_layers, batch, hidden_dim).

在本 notebook 示例中，这些概念将变得更加清晰。本 notebook 和 后续的 notebook 都是[此PyTorch LSTM教程](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html#lstm-s-in-pytorch)的修改版。

让我们举一个简单例子，例如我们想用 LSTM 处理一个句子 "Giraffes in a field"，将其输入 序列模型。我们的输入应是一个元素是单词的，"1x4"的行向量：

\begin{align}\begin{bmatrix}
   \text{Giraffes  } 
   \text{in  } 
   \text{a  } 
   \text{field} 
   \end{bmatrix}\end{align}

在本例中，输入语句有4个单词，我们要设定每个时间步要生成多少个输出，例如设定每个LSTM单元生成 **3个隐藏状态值**。将LSTM中的循环层数保持为默认值1。

隐藏状态和记忆单元的张量形状是（n_layers，batch，hidden_dim），在本例中，该张量形状是（1，1，3），即其输入是一批/单词序列（这一句话），循环层数为一层，生成3个隐藏状态值。

### 示例代码

接下来，让我们看一个LSTM的示例，该示例旨在查看输入是有4个值的序列（用数值类型，因为容易创建和跟踪），输出是3个值。这就是上面语句处理网络，建议你修改输入/隐藏状态的大小，以查看对LSTM结构有何影响！

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

%matplotlib inline

torch.manual_seed(2) # so that random variables will be consistent and repeatable for testing

### 定义一个简单的LSTM

**关于隐藏状态和输出的数量的说明**

除非你定义自己的LSTM，并通过在网络末端添加线性层来改变输出数量 例如，fc = nn.Linear(hidden_dim, output_dim)，否则隐藏状态数`hidden_dim`和输出的数量是相同的。

In [ ]:
from torch.autograd import Variable

# define an LSTM with an input dim of 4 and hidden dim of 3
# this expects to see 4 values as input and generates 3 values as output
input_dim = 4
hidden_dim = 3
lstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim)  

# make 5 input sequences of 4 random values each
inputs_list = [torch.randn(1, input_dim) for _ in range(5)]
print('inputs: \n', inputs_list)
print('\n')

# initialize the hidden state
# (1 layer, 1 batch_size, 3 outputs)
# first tensor is the hidden state, h0
# second tensor initializes the cell memory, c0
h0 = torch.randn(1, 1, hidden_dim)
c0 = torch.randn(1, 1, hidden_dim)


h0 = Variable(h0)
c0 = Variable(c0)
# step through the sequence one element at a time.
for i in inputs_list:
    # wrap in Variable 
    i = Variable(i)
    
    # after each step, hidden contains the hidden state
    out, hidden = lstm(i.view(1, 1, -1), (h0, c0))
    print('out: \n', out)
    print('hidden: \n', hidden)


你可以看到输出张量和隐藏张量的长度始终为3，因为在定义LSTM时，我们已经用`hidden_dim`指定了。

### 一次全部

for循环对于处理大的数据序列不是很有效，因此我们也可以 **一次处理所有这些输入。**

1. 将所有输入序列连接到一个大小为 batch_size 的大张量中
2. 定义隐藏状态的张量形状
3. 获取输出和*最近*的隐藏状态（即根据序列中的最新的单词创建）

由于初始隐藏状态不同，输出看起来可能略有不同。

In [ ]:
# turn inputs into a tensor with 5 rows of data
# add the extra 2nd dimension (1) for batch_size
inputs = torch.cat(inputs_list).view(len(inputs_list), 1, -1)

# print out our inputs and their shape
# you should see (number of sequences, batch size, input_dim)
print('inputs size: \n', inputs.size())
print('\n')

print('inputs: \n', inputs)
print('\n')

# initialize the hidden state
h0 = torch.randn(1, 1, hidden_dim)
c0 = torch.randn(1, 1, hidden_dim)

# wrap everything in Variable
inputs = Variable(inputs)
h0 = Variable(h0)
c0 = Variable(c0)
# get the outputs and hidden state
out, hidden = lstm(inputs, (h0, c0))

print('out: \n', out)
print('hidden: \n', hidden)

### 接下来：隐藏状态和门

该 notebook 向你展示了PyTorch中LSTM的输入和输出的结构。在接下来的notebook 练习中，你将了解有关LSTM如何利用隐藏状态表示长期和短期记忆的更多信息。

#### 词性
在本课后面的 notebook 中，你将看到如何定义一个模型来标记词性（名词，动词，限定词），包括定义一个LSTM和一个线性层，以定义所需的输出大小，
最后训练模型，以创建将一个将每个输入单词与词性相关联的分类得分分布。